# Tuning du modèle

L'objet de ce notebook est d'illustrer les différentes étapes de tuning du modèle.


## Préambule

### Imports

In [1]:
# setting up sys.path for relative imports
from pathlib import Path
import sys
project_root = str(Path(sys.path[0]).parents[1].absolute())
if project_root not in sys.path:
    sys.path.append(project_root)

In [118]:
# imports and customization of diplay
# import os
import re
from functools import partial
# import numpy as np
import pandas as pd
pd.options.display.min_rows = 6
pd.options.display.width=108
# from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
# from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
# from matplotlib import pyplot as plt

from src.pimest import ContentGetter
from src.pimest import PathGetter
from src.pimest import PDFContentParser
from src.pimest import BlockSplitter
from src.pimest import SimilaritySelector
# from src.pimest import custom_accuracy
from src.pimest import text_sim_score
# from src.pimest import text_similarity
# from src.pimest import build_text_processor

### Acquisition des données

On récupère les données manuellement étiquetées et on les intègre dans un dataframe

In [107]:
ground_truth_df = pd.read_csv(Path('..') / '..' / 'ground_truth' / 'manually_labelled_ground_truth.csv',
                              sep=';',
                              encoding='latin-1',
                              index_col='uid')
ground_truth_uids = list(ground_truth_df.index)

acqui_pipe = Pipeline([('PathGetter', PathGetter(ground_truth_uids=ground_truth_uids,
                                                  train_set_path=Path('..') / '..' / 'ground_truth',
                                                  ground_truth_path=Path('..') / '..' / 'ground_truth',
                                                  )),
                        ('ContentGetter', ContentGetter(missing_file='to_nan')),
                        ('ContentParser', PDFContentParser(none_content='to_empty')),
                       ],
                       verbose=True)

texts_df = acqui_pipe.fit_transform(ground_truth_df)
texts_df['ingredients'] = texts_df['ingredients'].fillna('')
texts_df

[Pipeline] ........ (step 1 of 3) Processing PathGetter, total=   0.1s
[Pipeline] ..... (step 2 of 3) Processing ContentGetter, total=   0.1s
Launching 8 processes.
[Pipeline] ..... (step 3 of 3) Processing ContentParser, total=  38.3s


### Train / Test split

On va appliquer une grid search pour déterminer les meilleurs paramètres de notre modèle. 
Pour ne pas surestimer la performance du modèle, il est nécessaire de bien séparer le jeu de test du jeu d'entraînement, y compris pour la grid search !

In [137]:
train, test = train_test_split(texts_df, test_size=100, random_state=42)

Dans toute la suite, on utilisera le jeu d'entraînement pour effectuer le tuning des hyperparamètres.

## Ajustement de la fonction de découpage des textes

L'objectif de cette partie est d'optimiser la fonction de découpage des textes en blocs. On va tester quelques fonctions candidates, via une GridSearch.

### Définition des fonctions candidates

On définit les fonctions de split : 

In [138]:
# definitions of splitter funcs
splitter_funcs = []
def split_func(text):
    return(text.split('\n\n'))
splitter_funcs.append(split_func)
def split_func(text):
    return(text.split('\n'))
splitter_funcs.append(split_func)
def split_func(text):
    regex = r'\s*\n\s*\n\s*'
    return(re.split(regex, text))
splitter_funcs.append(split_func)

### Mise en place du pipeline

On construit ensuite un pipeline de traitement du texte.
Le SimilaritySelector prenant en entrée une pandas.Series, on définit entre le BlockSplitter (dont la méthode transform() retourne un pandas.DataFrame) et le SimilaritySelector une fonction utilitaire qui séléctionne la colonne 'blocks'.

In [139]:
def select_col(df, col_name='blocks'):
        return(df[col_name].fillna(''))
col_selector = FunctionTransformer(select_col)    

In [140]:
process_pipe = Pipeline([('Splitter', BlockSplitter()),
                         ('ColumnSelector', col_selector),
                         ('SimilaritySelector', SimilaritySelector())
                        ],
                       verbose=False)

On peut tester le fonctionnement de ce Pipeline.
Attention, les résultats ne sont pas représentatifs, on entraîne et on prédit sur le même jeu de données !

In [141]:
process_pipe.fit(train, train['ingredients'])
process_pipe.predict(train)

Launching 8 processes.
Launching 8 processes.


uid
02d5ceb9-21c2-4965-8f65-309bca7638b2    Café chicorée solubles et fibres de chicorée.\...
bbe72396-6ed4-4df1-935b-0c0a7dbd77dc                                                     
507b428e-e99d-464b-b9d3-50629efe4355    COMPOSITION\nMélange de Blés de pays recommand...
                                                              ...                        
4b28bb17-1f1d-4cbb-ac3b-80227ef248ab    Gluten\nCrustacés\nOeufs\nPoisson\nSoja\nLait\...
d2137dae-ff21-46ec-83be-7400773c6c3b    Amidon modifié de pomme de terre - Fécule de p...
571d98ae-9647-4bd4-ad1a-a497f93987cb    Composition typique (Données inappropriées pou...
Length: 400, dtype: object

### Application de la GridSearch

On applique ensuite une grid search en faisant varier les fonctions de split.

In [142]:
lev_scorer = partial(text_sim_score, similarity='levenshtein')

In [143]:
param_grid = [{'Splitter__splitter_func': splitter_funcs}]
search = GridSearchCV(process_pipe,
                      param_grid,
                      cv=8, 
                      scoring= lev_scorer,
                      n_jobs=-1,
                      verbose=0,
                     ).fit(train, train['ingredients'])

Launching 8 processes.


In [144]:
search.cv_results_

{'mean_fit_time': array([0.4982658 , 0.39767313, 0.2000843 ]),
 'std_fit_time': array([0.0765331 , 0.11518912, 0.00948586]),
 'mean_score_time': array([0.39683539, 0.36049438, 0.21251634]),
 'std_score_time': array([0.03982759, 0.09187401, 0.01686873]),
 'param_Splitter__splitter_func': masked_array(data=[<function split_func at 0x7f805cb67dc0>,
                    <function split_func at 0x7f805cb67820>,
                    <function split_func at 0x7f805cb678b0>],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'Splitter__splitter_func': <function __main__.split_func(text)>},
  {'Splitter__splitter_func': <function __main__.split_func(text)>},
  {'Splitter__splitter_func': <function __main__.split_func(text)>}],
 'split0_test_score': array([0.49546862, 0.36968256, 0.51266659]),
 'split1_test_score': array([0.57136762, 0.42644881, 0.60169897]),
 'split2_test_score': array([0.3800339 , 0.33117854, 0.4143086 ]),
 'split3_test_sc

On voit que la fonction de split la plus efficace est la fonction qui applique la regex (deux retours chariots parmi des whitespaces).

In [145]:
for i in range(len(search.cv_results_['rank_test_score'])):
    str_result = f"{search.cv_results_['mean_test_score'][i]:.2%} +/- {search.cv_results_['std_test_score'][i]:.2%}"
    print(str_result)

50.20% +/- 6.25%
38.40% +/- 4.50%
52.93% +/- 6.54%
